In [10]:
# ---- Install dependencies (only run once in Colab) ----
!pip install python-docx PyPDF2 pytesseract Pillow

In [ ]:
# ---- Imports ----
from google.colab import files
from io import BytesIO
import docx
from PIL import Image
import pytesseract
from PyPDF2 import PdfReader
import re

# ---- Extractors ----
def extract_text_from_pdf(file_bytes):
    reader = PdfReader(BytesIO(file_bytes))
    text = ""
    for page in reader.pages:
        text += page.extract_text() or ""
    return text

def extract_text_from_image(file_bytes):
    image = Image.open(BytesIO(file_bytes))
    return pytesseract.image_to_string(image)

def extract_text_from_docx(file_bytes):
    doc = docx.Document(BytesIO(file_bytes))
    return "\n".join([para.text for para in doc.paragraphs])

# ---- Resume Analyzer ----
def analyze_resume(text):
    required_fields = ['Education', 'Experience', 'Skills', 'Contact']
    feedback = {}
    lower_text = text.lower()

    feedback['Education'] = 'education' in lower_text
    feedback['Experience'] = 'experience' in lower_text
    feedback['Skills'] = 'skills' in lower_text

    # Check for 'contact' keyword OR phone number
    phone_pattern = r'\b(\+?\d{1,3}[-.\s]?)?(\(?\d{2,4}\)?[-.\s]?){2,5}\d{2,4}\b'
    has_phone = re.search(phone_pattern, text)
    feedback['Contact'] = 'contact' in lower_text or bool(has_phone)

    score = (sum(feedback.values()) / len(feedback)) * 100
    return score, feedback

# ---- Step 1: Upload file ----
uploaded = files.upload()
file_name = list(uploaded.keys())[0]
file_bytes = uploaded[file_name]

# ---- Step 2: Extract resume text ----
resume_text = ""

try:
    if file_name.lower().endswith('.pdf'):
        resume_text = extract_text_from_pdf(file_bytes)
    elif file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        resume_text = extract_text_from_image(file_bytes)
    elif file_name.lower().endswith('.docx'):
        resume_text = extract_text_from_docx(file_bytes)
    else:
        print(f"⚠️ No extractor available for '{file_name}'. Proceeding without parsing.")
except Exception as e:
    print(f"❌ Error processing file: {e}")

# ---- Step 3: Analyze resume ----
if resume_text:
    score, feedback = analyze_resume(resume_text)

    print(f"\n✅ Resume Match Score: {score:.2f}%\n")

    if score == 100:
        print("🌟 Excellent Resume! All important fields are present.\n✅ Ready for applications and interviews!")
    else:
        print("⚠️ Resume is missing the following sections:")
        for field, is_present in feedback.items():
            if not is_present:
                print(f"🔸 {field}")
        print("\n📌 Suggestion: Add the above missing sections to improve your resume quality.")

# ---- Optional: Debug print ----
# print("\n--- Extracted Resume Text ---\n")
# print(resume_text)
